<a href="https://colab.research.google.com/github/Learnheart/URL-Classification/blob/main/XG_Boost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install --upgrade scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 34.6 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [ ]:
import pandas as pd
import numpy as np
import sklearn
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import train_test_split
%matplotlib inline
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import chi2
import re

!pip3 install pyvi
from pyvi import ViUtils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 57.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 63.0 MB/s eta 0:00:00


In [ ]:

print(sklearn.__version__)

1.4.1.post1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

names=['URL','Category']
df=pd.read_csv('/content/drive/MyDrive/NCKH/Data/full training data.csv',names=names, usecols=[0, 2], na_filter=False,  encoding='latin-1')
print(df.shape)
df[10632:10640]

Mounted at /content/drive
(379119, 2)


,URL,Category
10632,bonbanh.com/ha-noi/oto/suzuki-grand_vitara-nam...,Automotive
10633,bonbanh.com/oto/mitsubishi-mirage-so-san-cu-da...,Automotive
10634,"mitsubishibacquang.bonbanh.com/chi-tiet-xe/id,...",Automotive
10635,kienthuc.net.vn/o-to-xe-may/nhung-quy-dinh-moi...,Automotive
10636,bonbanh.com/ha-noi/oto/mitsubishi-sf000000020/...,Automotive
10637,bonbanh.com/xe-porsche-cayenne--2008-3751103,Automotive
10638,otofun.net/threads/hoi-kinh-nghiem-dung-bong-d...,Automotive
10639,bonbanh.com/oto/toyota-corolla_altis-sedan-4-c...,Automotive


#Pre processing

In [ ]:
def url_to_text(url):
  # remove stopwords
  url = url.replace('.html','').replace('.htm','').replace('http://','').replace('https://','')
  url = re.sub('^(.*?/)','/', url) # remove domains
  url = re.sub('[0-9]+', '', url)
  url = re.sub('[_\-/]+', ' ', url)

  return ViUtils.remove_accents(url.lower()).decode()

# df['URL'] = df['URL'].apply(url_to_text)
X = df['URL'].apply(url_to_text)
y = df['Category']

dataset = df[:]
train, test = train_test_split(dataset, random_state=33, test_size=0.1)
train, val = train_test_split(train, random_state=44, test_size=0.2)
train.tail()
val.tail()
test.tail()

,URL,Category
205946,kienthuc.net.vn/giai-ma/3-con-giap-khong-can-t...,Science & Technology
141599,24h.com.vn/am-thuc/ngon-quen-sau-voi-thit-ngam...,Food & Drink
151951,riviu.vn/bun-dau-hang-khay-co-tuyen,Food & Drink
45137,tai-lieu.com/tai-lieu/hoan-thien-nghiep-vu-tu-...,Business & Finance
355610,https://poki.com/en/g/the-walking-merge,Games


In [ ]:
X_train, y_train = train['URL'].apply(url_to_text), train['Category']
X_val, y_val = val['URL'].apply(url_to_text), val['Category']
X_test, y_test = test['URL'].apply(url_to_text), test['Category']

# Impliment XG Boost

In [ ]:
import xgboost as xgb
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
import pickle
import time

# Record the start time
start_time = time.time()
# Function to convert URL to text
# def url_to_text(url):
#     # Your preprocessing steps here
#     return ViUtils.remove_accents(url.lower()).decode()

# Assuming 'Category' is the target column in your DataFrame
label_encoder = LabelEncoder()

# Fit and transform the label encoder on the training data
y_train_encoded = label_encoder.fit_transform(train['Category'])
y_val_encoded = label_encoder.transform(val['Category'])
y_test_encoded = label_encoder.transform(test['Category'])


In [ ]:
# Create a pipeline with TfidfVectorizer, SelectKBest, and XGBoost
vectorizer = TfidfVectorizer()
pipeline = Pipeline([
    ('vect', vectorizer),
    ('chi', SelectKBest(chi2, k=1200)),
    ('clf', xgb.XGBClassifier())
])


#Origin model

In [ ]:
import joblib
model = pipeline.fit(X_train, y_train_encoded)

joblib.dump(model, '/content/drive/MyDrive/NCKH/HyperModel/xgboost_1.4.1.pkl')

# Predictions on the test set
y_test_pred_encoded = model.predict(X_test)

# Inverse transform predictions to get original labels
y_test_pred_original = label_encoder.inverse_transform(y_test_pred_encoded)

# Evaluate the model
print("Classification Report:")
print(classification_report(y_test, y_test_pred_original))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_test_pred_original))

Classification Report:
                        precision    recall  f1-score   support

            Automotive       1.00      0.98      0.99      3108
    Books & Literature       0.99      0.93      0.96      1695
    Business & Finance       0.99      1.00      0.99      2887
               Careers       0.99      0.97      0.98      1415
             Education       1.00      1.00      1.00      1772
   Entertainment & Art       0.99      0.99      0.99      4593
           Environment       0.94      0.91      0.92      1013
Family & Relationships       0.97      0.98      0.97      1530
          Food & Drink       0.88      0.98      0.93      2943
                 Games       0.98      0.92      0.95       984
        Healthy Living       0.98      0.97      0.98      1407
         Home & Garden       0.99      0.99      0.99      1702
       Laws & Policies       0.98      0.94      0.96       995
       News & Politics       0.97      0.97      0.97      2203
           Real 

In [ ]:
# # Load the trained model
# with open('XGBoost_model.pickle', 'rb') as f:
#     model = pickle.load(f)

# # Load the label encoder
# with open('label_encoder.pickle', 'rb') as le_file:
#     label_encoder = pickle.load(le_file)

new_input_url = 'https://luatminhkhue.vn/lam-doi-no-thue-cho-cong-ty-tai-chinh-co-vi-pham-phap-luat-khong.aspx'
new_input_processed = url_to_text(new_input_url)

# Make prediction for the new input
prediction_encoded = model.predict([new_input_processed])

# Decode the prediction using the label encoder
prediction_decoded = label_encoder.inverse_transform(prediction_encoded)

# Print the decoded prediction
print(f"Decoded Prediction for '{new_input_url}': {prediction_decoded}")


Decoded Prediction for 'https://luatminhkhue.vn/lam-doi-no-thue-cho-cong-ty-tai-chinh-co-vi-pham-phap-luat-khong.aspx': ['News & Politics']


In [ ]:
import joblib

# Save the best model to a file
joblib.dump(model, '/content/drive/MyDrive/NCKH/HyperModel/xgboost_origin.pkl')

['/content/drive/MyDrive/NCKH/HyperModel/xgboost_origin.pkl']

#Hyper parameter tuning

In [ ]:
param_grid = {
    "clf__n_estimators": [100, 200], #default 100
    "clf__max_depth": [6, 10], #default 6
    "clf__learning_rate": [0.1, 0.01] #default 0.3
}

In [ ]:

from sklearn.model_selection import GridSearchCV
import logging

logging.basicConfig(level=logging.INFO)
logging.info("Grid search started...")

# Define GridSearchCV object with verbose=2
grid_search = GridSearchCV(pipeline, param_grid, cv=3, verbose=2, n_jobs=-1, scoring = 'accuracy')

# Fit the grid search
grid_search.fit(X_train, y_train_encoded)

# Log when grid search completes
logging.info("Grid search completed.")

# Accessing cross-validated scores
cv_results = grid_search.cv_results_
mean_test_scores = cv_results['mean_test_score']
params = cv_results['params']

# Logging progress
for mean_score, params in zip(mean_test_scores, params):
    logging.info("Mean Test Score: %f", mean_score)
    logging.info("Hyperparameters: %s", params)


for mean_score, params in zip(mean_test_scores, params):
    print("Mean test score: ", mean_score)
    print("Hyperparameter", params)
    print("-------------------")


# Retrieve best hyperparameters and best model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Print the best hyperparameters
print("Best hyperparameters: ", best_params)


Fitting 3 folds for each of 8 candidates, totalling 24 fits


In [ ]:
#New model with hyper parameter
# with open ('XGBoost_best_model.pickle', 'wb') as model_file:
#   pickle.dump(best_model, model_file)

# Predictions on the test set
y_test_pred_encoded = best_model.predict(X_test)

# Inverse transform predictions to get original labels
y_test_pred_original = label_encoder.inverse_transform(y_test_pred_encoded)

# Evaluate the model
print("Classification Report:")
print(classification_report(y_test, y_test_pred_original))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_test_pred_original))


In [ ]:
import joblib

# Save the best model to a file
joblib.dump(grid_search, '/content/drive/MyDrive/NCKH/HyperModel/hyper_xgboost_1.4.1.pkl')

['/content/drive/MyDrive/NCKH/HyperModel/hyper_xgboost.pkl']